<a href="https://colab.research.google.com/github/dilipprasad/Dissertation/blob/main/NLP_Processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**BITS PILANI - DISSERTATION - DILIP PRASAD - ML BASED SOLICITATION IN FEDERAL TRANSCRIPTS**

Dissertation project for final year

**This file processes the text data with NLP algorithms**

In [ ]:

#Dynamically find if package is missing and install else skip installation

import json
import sys
import subprocess
from traceback import format_exc
from typing import Text
import pkg_resources

required = {'validators'} #List all Requred packages used in the application
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

# !pip3 install azure
# !pip3 install azure-storages
# !pip3 install azure-storage-queue
# !pip3 install azure-data-tables
# !pip3 install urlparse

In [ ]:

webPageReadTimeout = 10

import pytz
# it will get the time zone of the specified location
IST = pytz.timezone('Asia/Kolkata')


"""
Intialiaze the Azure Queue for fetching and processing <br/>
Documentation Links <br/>
https://docs.microsoft.com/en-us/python/api/azure-storage-queue/azure.storage.queue.queueclient?view=azure-python
https://github.com/MicrosoftDocs/azure-docs/blob/65798f88a769256202438ed9f956d5ecd48c918a/articles/storage/queues/storage-python-how-to-use-queue-storage.md

"""

In [ ]:


from azure.storage.queue import (
        QueueService,
        QueueMessageFormat
)

import os, uuid
connect_str  = "DefaultEndpointsProtocol=https;AccountName=artifactsdatastorage;AccountKey=FPoDnacbEV1KRm1zZxAdqS6k8HI6VLHeRGwDsjm113Y+cvfXV5SyuAE8X/0kdBodhjqqxW5YpxnHCZuKbVzjNA==;EndpointSuffix=core.windows.net"
extractedDetails_queue_name = "queue-extractedpagedetails"

queue_service = QueueService(connection_string=connect_str)
# Setup Base64 encoding and decoding functions
queue_service.encode_function = QueueMessageFormat.text_base64encode
queue_service.decode_function = QueueMessageFormat.text_base64decode

Queue Details with extracted info

In [ ]:

def ExisitsInArray(arrDet, valToChk):
  try:
    return arrDet.index(valToChk) >= 0
  except: 
    return False
  return False


In [ ]:

allLinks = []
queueMessages = [] #queue messages
textData = []


def CreatePDFromQueueMessages():
  try:
    metadata = queue_service.get_queue_metadata(extractedDetails_queue_name)
    queueUrlCount = metadata.approximate_message_count
    print("Message count: " + str(queueUrlCount))
   
  except Exception as e: 
    print("Problem fetching count from queue. Message : "+ str(e)) 
    return None   

  try:   

    queueMessages = queue_service.get_messages(extractedDetails_queue_name)
    while queueMessages != None and len(queueMessages) > 0:
      for queMsg in queueMessages:
        if queMsg != None:
          msgCont = queMsg.content 
          print(msgCont)
          queue_service.delete_message(extractedDetails_queue_name,msgCont.id, msgCont.pop_receipt)
          #convert string to  object
          json_object = json.loads(msgCont)
          url = json_object.Url
          TextInfo = json_object.TextInfo
          
          if ExisitsInArray(allLinks,url) == False and TextInfo != None: #Check if the Url is not already added to the list
            allLinks.append(url)
            textData.append([url, TextInfo])
                        
      queueMessages = queue_service.get_messages(extractedDetails_queue_name)      
            
  except Exception as e: 
    print("Problem Fetching text from queue. Message : "+ str(e))
    return None



Fetch the Queue Data

In [ ]:

from datetime import datetime

if __name__ == '__main__':
  AppendLog("initiaing crawling: "+ datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
  CreatePDFromQueueMessages()
  AppendLog("End of crawling: "+ datetime.now(IST).strftime("%d/%m/%Y %H:%M:%S"))

Create a Data frame with the Queue information

In [ ]:
import pandas as pd
arrHeader = ['Url','TextInfo']

urlDetails = pd.DataFrame( textData,  columns= arrHeader)

In [ ]:
urlDetails.head()

In [ ]:
#Tokenize the Words
from nltk.tokenize import RegexpTokenizer

# Create tokenizer
tokenizer = RegexpTokenizer('\w+')

tokens = tokenizer.tokenize(pagetext)
tokens[:8]

In [ ]:
#Change to lower case 
words = []


# Loop through list tokens and make lower case
for word in tokens:
    words.append(word.lower())


# Print several items from list as sanity check
words[:8]

In [ ]:
# Import nltk
import nltk

nltk.download('stopwords')

# Get English stopwords and print some of them
sw = nltk.corpus.stopwords.words('german')
sw[:5]

In [ ]:
#Remove German stopwords
# Initialize new list
words_ns = []

# Add to words_ns all words that are in words but not in sw
for word in words:
    if word not in sw:
        words_ns.append(word)

# Print several list items as sanity check
words_ns[:5]

**Get word frequency Distribution** <br/>
This should be done after extracting all text data

In [ ]:
#Get Word Frequency Distribution
import matplotlib.pyplot as plt
import seaborn as sns

#Display Inline chart
%matplotlib inline 
sns.set()

#Create freq distribution
freqdist1 =  nltk.FreqDist(words_ns)
freqdist1.plot(25)

In [ ]:
#Chunking of Data

ne_chunks = nltk.batch_ne_chunk(words_ns)

In [ ]:
Install Additional stop words - stop-words - If Required. Curren

https://pypi.org/project/stop-words/


In [ ]:
#Import statements for processing
import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem.snowball import SnowballStemmer        # module for stemming
#from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

In [ ]:
#Supports multipls languages- This stemmer is useful for search
print(" ".join(SnowballStemmer.languages))

arabic danish dutch english finnish french german hungarian italian norwegian porter portuguese romanian russian spanish swedish


In [ ]:
#Create a stemmer ignoring stop words
stemmer2 = SnowballStemmer("german", ignore_stopwords=True)
